In [43]:
import pandas as pd
import numpy as np
import math
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn. preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

In [44]:
diamonds = pd.read_csv('diamonds_train.csv')
predict = pd.read_csv('diamonds_predict.csv')

In [45]:
diamonds = diamonds[(diamonds['x'] > 0) & (diamonds['y'] > 0) & (diamonds['z'] > 0)]

In [46]:
diamonds['x'] = np.log(diamonds['x'])
diamonds['y'] = np.log(diamonds['y'])
diamonds['z'] = np.log(diamonds['z'])

predict['x'] = np.log(predict['x'])
predict['y'] = np.log(predict['y'])
predict['z'] = np.log(predict['z'])

/home/ana/miniconda3/envs/ironhack_env/lib/python3.7/site-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [47]:
# Transformation which relates x, y and z with diamond volume: CONE

diameter = (diamonds['x'] + diamonds['y']) / 2
diamonds['volume cone'] = (1/3) * np.pi * ((diameter / 2) ** 2) * diamonds['z']
diamonds['volume semisphere'] = (4/6) * np.pi * ((diameter / 2)) **2
diamonds['volume'] = diamonds['volume cone'] + diamonds['volume semisphere']

In [48]:
diameter_predict = (predict['x'] + predict['y']) / 2
predict['volume cone'] = (1/3) * np.pi * ((diameter_predict / 2) ** 2) * predict['z']
predict['volume semisphere'] = (4/6) * np.pi * ((diameter_predict / 2)) **2
predict['volume'] = predict['volume cone'] + predict['volume semisphere']

In [92]:
# Transformation which calculate the price due to price per carat * carat weight

diamonds_price = []

for i in diamonds['carat']:
    
    if i < 0.5:
        diamonds_price.append(600)        
    elif i >= 0.5 and  i < 1:
        diamonds_price.append(2400)
    elif i >= 1 and i < 1.5:
        diamonds_price.append(4280)
    elif i>= 1.5 and i < 2:
        diamonds_price.append(9360)
    else: 
        diamonds_price.append(16000)

In [93]:
diamonds['carat price'] = diamonds_price

In [94]:
predict_price = []

for i in predict['carat']:
    
    if i < 0.5:
        predict_price.append(600)        
    elif i >= 0.5 and  i < 1:
        predict_price.append(2400)
    elif i >= 1 and i < 1.5:
        predict_price.append(4280)
    elif i>= 1.5 and i < 2:
        predict_price.append(9360)
    else: 
        predict_price.append(16000)

In [95]:
predict['carat price'] = predict_price

In [96]:
diamonds['predicted price'] = diamonds['carat'] * diamonds['carat price']

In [97]:
predict['predicted price'] = predict['carat'] * predict['carat price']

In [98]:
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z,volume cone,volume semisphere,volume,carat price,predicted price
0,1.21,Premium,J,VS2,62.4,58.0,4268,1.921325,1.915451,1.446919,1.394073,1.926954,3.321028,4280,5178.8
1,0.32,Very Good,H,VS2,63.0,57.0,505,1.470176,1.477049,1.011601,0.575101,1.137012,1.712113,600,192.0
2,0.71,Fair,G,VS1,65.5,55.0,2686,1.726332,1.710188,1.294727,1.000748,1.545882,2.546629,2400,1704.0
3,0.41,Good,D,SI1,63.8,56.0,738,1.543298,1.551809,1.098612,0.688818,1.253978,1.942796,600,246.0
4,1.02,Ideal,G,SI1,60.5,59.0,4882,1.879465,1.873339,1.373716,1.266244,1.843531,3.109775,4280,4365.6


In [99]:
diamonds.describe()

,carat,depth,table,price,x,y,z,volume cone,volume semisphere,volume,carat price,predicted price
count,40439.000000,40439.000000,40439.000000,40439.000000,40439.000000,40439.000000,40439.000000,40439.000000,40439.000000,40439.000000,40439.000000,40439.000000
mean,0.797425,61.753013,57.445543,3926.535448,1.726726,1.727411,1.244759,1.018150,1.581374,2.599524,3329.492816,4208.743342
std,0.475257,1.431306,2.233055,3990.024501,0.194158,0.193063,0.194397,0.380567,0.352796,0.732688,3492.792584,7269.656120
min,0.200000,43.000000,43.000000,326.000000,1.327075,1.313724,0.067659,0.063531,0.912871,1.295020,600.000000,120.000000
25%,0.400000,61.000000,56.000000,945.000000,1.549688,1.551809,1.068153,0.674547,1.259154,1.934380,600.000000,240.000000
50%,0.700000,61.800000,57.000000,2396.000000,1.738710,1.742219,1.258461,1.000263,1.586086,2.587385,2400.000000,1680.000000
75%,1.040000,62.500000,59.000000,5329.500000,1.877937,1.877937,1.396245,1.279291,1.845044,3.123548,4280.000000,4451.200000
max,4.500000,79.000000,95.000000,18823.000000,2.325325,4.075841,2.086914,5.193813,4.977507,10.171320,16000.000000,72000.000000


In [100]:
# The first step is defining the Data Cleaning and Feature Engineering 

target = 'price'
cat_features = ['cut', 'color', 'clarity']
num_features = ['carat', 'depth', 'table', 'x', 'y', 'z', 'volume cone', 'volume semisphere', 'volume','predicted price']
features = cat_features + num_features

## **Random Forest**

In [101]:
# Generamos una primera etapa del pipeline que transforma los datos categoricos y los datos numéricos del modelo

# En los datos numéricos fill los nulos con la mediana y estandarizamos los resultados
numeric_transformer = \
Pipeline(steps = [('imputer', SimpleImputer(strategy = 'median')), ('scalar', StandardScaler())])

# En los datos categóricos fill los nulos con una constante y los transformamos con OneHotEncoding
categorical_transformer = \
Pipeline(steps = [('imputer', SimpleImputer(strategy = 'constant', fill_value = 'missin')), 
                  ('onehot', OneHotEncoder(handle_unknown = 'ignore'))])

In [102]:
# Generamos una segunda etapa donde definimos las variables numericas a las que aplicar numeric_transformer y las variables categoricas a las que aplicar categorical_transformer

preprocessor = \
ColumnTransformer(transformers = [('num', numeric_transformer, num_features), ('cat', categorical_transformer, cat_features)])

In [103]:
# Generamos una última etapa donde definimos el modelo

model = \
Pipeline(steps = [('preprocessor', preprocessor), ('regressor', RandomForestRegressor(n_estimators = 150, max_depth = 10, random_state = 5))])

In [104]:
# Dividimos el dataset en train y en test

diamonds_train, diamonds_test = train_test_split(diamonds)

In [105]:
model.fit(diamonds_train[features], diamonds_train[target])

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scalar',
                                                                   StandardScaler())]),
                                                  ['carat', 'depth', 'table',
                                                   'x', 'y', 'z', 'volume cone',
                                                   'volume semisphere',
                                                   'volume',
                                                   'predicted price']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                  

In [106]:
# Validamos el modelo con diamonds_test

y_test_pred = model.predict(diamonds_test[features])
y_test_real = diamonds_test[target]

mean_squared_error(y_test_real, y_test_pred, squared = False)

661.9011165123562

In [107]:
# Validamos el modelo con diamonds_train

y_train_pred = model.predict(diamonds_train[features])
y_train_real = diamonds_train[target]

mean_squared_error(y_train_real, y_train_pred, squared = False)

595.726220301312

## **Submission**

In [19]:
submission_df = pd.DataFrame({'id': predict['id'], 'price': y_pred})

In [20]:
submission_df.to_csv('./data/GradientBoosting.csv', index = False)

## **Hyperparameter Optimization**

In [17]:
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],
    'regressor__n_estimators': [16, 32, 64, 128, 256, 512],
    'regressor__max_depth': [2, 4, 8, 16, 20],
}

grid_search = RandomizedSearchCV(model, 
                                 param_grid, 
                                 cv=5, 
                                 verbose=10, 
                                 scoring='neg_root_mean_squared_error', 
                                 n_jobs=-1,
                                 n_iter=32)

grid_search.fit(diamonds[features], diamonds[target])

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   18.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   44.1s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed: 10.1min finished


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median')),
                                                                                               ('scalar',
                                                                                                StandardScaler())]),
                                                                               ['carat',
                                                                                'depth',
                                                                                'table',
                                                                                'x',
              

In [18]:
grid_search.best_params_

{'regressor__n_estimators': 512,
 'regressor__max_depth': 16,
 'preprocessor__num__imputer__strategy': 'mean'}

In [19]:
grid_search.best_score_

-557.7769864705173

In [22]:
y_pred = grid_search.predict(predict[features])

In [23]:
# Validamos el modelo con diamonds_test

y_test_pred = model.predict(diamonds_test[features])
y_test_real = diamonds_test[target]

mean_squared_error(y_test_real, y_test_pred, squared = False)

623.4305379967942

## **Submission**

In [24]:
submission_df = pd.DataFrame({'id': predict['id'], 'price': y_pred})

In [25]:
submission_df.to_csv('./data/RandomForest.csv', index = False)